In [2]:
import itertools
import pandas as pd
from collections import defaultdict
from itertools import islice, chain
import re
from datetime import datetime
from datetime import timedelta
import networkx as nx
import numpy as np
import os

In [14]:
# res = pd.read_csv('0_1000_proto.csv')
# order = ['head','tail','sentence','time','head_code','tail_code']
# temp = res[order]
# temp

In [15]:
# data = []
# with open('temp_train.txt', encoding='utf-8', mode = 'r') as f:
#     for line in f:
#         data.append(line)


In [16]:
# kg = []
# company_ls = []
# for i in range(len(data)):
#     head,tail,relation,sentence = data[i].split('\t')
#     kg.append((temp['head_code'][i],temp['tail_code'][i],relation,temp['time'][i]))

In [17]:
# kg_df = pd.DataFrame(kg,columns=['head','tail','relation','time'])

# dataloader for new data

In [3]:
data = pd.read_csv('company_relation.csv')
kg = data.drop(['Unnamed: 0'],axis =1)
kg

,head,tail,relation,sentence,time,head_code,tail_code
0,上海机场,白云机场,same_industry,继续推荐国际旅客占比高、免税经营更为灵活的上海机场，关注产能空间持续打开，商业消费有望迎高成...,2022-02-14 08:58:14,600009.SH,600004.SH
1,赣锋锂业,中铝国际,rise,有色金属板块集体上涨，赣锋锂业涨7%，中铝国际涨超4%,2022-02-17 12:08:27,002460.SZ,601068.SH
2,贵州茅台,宁德时代,compete,贵州茅台超过宁德时代，重回公募基金头号重仓股,2022-01-25 17:10:52,600519.SH,300750.SZ
3,瑞玛工业,拓普集团,unknown,瑞玛工业在互动平台表示：公司精密紧固件产品、精密冲压零部件产品通过拓普集团、采埃孚天合、哈曼...,2021-12-10 21:59:00,2976,601689.SH
4,宁德时代,海通证券,unknown,宁德时代今日出现8.367亿元平价大宗交易，成交价每股569.13元，海通证券上海余姚路卖出...,2022-01-20 16:27:54,300750.SZ,600837.SH
...,...,...,...,...,...,...,...
8596,东风汽车,亿华通,same_industry,公司客户有东风汽车集团、吉利汽车、柳州五菱、PLUG，POWER、亿华通、上海重塑、康明...,2021-08-09 16:15:00,600006.SH,688339
8597,上汽集团,比亚迪,unknown,据悉，有投资者向上汽集团提问，请问懂秘，看到网上传言比亚迪将收购上汽乘用车，请问这是否属实？...,2021-08-09 13:34:00,600104.SH,002594.SZ
8598,洛阳钼业,中国铝业,rise,南方锰业涨13.7%，洛阳钼业涨6.5%，兴发铝业涨1.1%，中国铝业涨0.4%,2021-08-09 13:11:00,603993.SH,601600.SH
8599,星网锐捷,润和软件,cooperate,星网锐捷消息，近日，星网锐捷旗下升腾资讯与润和软件达成战略合作，发布全球首款基于OpenHa...,2021-08-09 13:06:00,002396.SZ,300339.SZ


In [19]:
# data1 = pd.read_csv('new_annotated_relation.csv')
# data2 = pd.read_csv('new_company_relation.csv',sep='\t')
# data3 = pd.read_csv('company_relation_20211106.csv',sep='\t')
# all_data = pd.concat([data1,data2,data3],axis=0,ignore_index=True)
# # order = ['head','tail','relation','sentence','time','head_code','tail_code']
# # kg_df = pd.DataFrame(all_data,columns=order)
# # kg_df = all_data.drop(['Unnamed: 0','head','tail','sentence'],axis =1)
# kg_df = all_data.drop(['Unnamed: 0'],axis =1)
# # kg_df
# kg_df

In [20]:
# relation_dict = {
#     '同行':'same_industry',
#     '同涨':'rise',
#     '竞争':'compete',
#     '未知Unknown':'unknown', 
#     '合作':'cooperate', 
#     '增持':'increase_holding', 
#     '同跌':'fall', 
#     '供应':'supply', 
#     '被减持':'be_reduced_holding',
#     '被投资':'be_invested', 
#     '减持':'reduce_holding', 
#     '上级':'superior', 
#     '被增持':'be_increased_holding', 
#     '下级':'subordinate', 
#     '投资':'invest', 
#     '纠纷':'dispute', 
#     '被供应':'be_supplied'
# }

# kg = kg_df.replace({"relation": relation_dict})

# links

## graph for each relation

In [21]:
# relation_ls = kg['relation'].unique()

In [22]:
# for r in relation_ls:
#     subgraph = kg[kg['relation'].isin([r])]
#     subgraph = subgraph.drop(['relation'], axis=1)
#     subgraph = subgraph.sort_values(['time'])
#     subgraph = subgraph.reset_index()
#     exec("subgraph_%s=subgraph"%r)
#     print(r,len(subgraph))

## timestamp

In [4]:
import dateutil.parser

def getDateTimeFromISO8601String(s):
    d = dateutil.parser.parse(s)
    d.strftime('%Y%m%d%H%M')
    return d

In [6]:
def build_links(kg_df):    
    links = []
    ts = []
    for i in range(len(kg_df)):
        timesting = kg_df['time'][i]
        timestamp = getDateTimeFromISO8601String(timesting)
        ts.append(timestamp)
        links.append((kg_df['head_code'][i],kg_df['relation'][i],kg_df['tail_code'][i],timestamp))
    return links,ts

# build subgraph

In [25]:
def subgraph(links,ts,SLICE_MONTHS,START_DATE,MAX_DATE):
    slices_links = defaultdict(lambda :{})
    slices_features = defaultdict(lambda : {})

    print ("Start date", START_DATE)
    slice_id = 0

    # Split the set of links in order by slices to create the graphs.
    for (a, b, time) in links:
        prev_slice_id = slice_id

        datetime_object = time
        if datetime_object > MAX_DATE:
            months_diff = (MAX_DATE - START_DATE).days//7  # 
        else:
            months_diff = (datetime_object - START_DATE).days//7
        slice_id = months_diff // SLICE_MONTHS
        slice_id = max(slice_id, 0)

        if slice_id == 1+prev_slice_id and slice_id > 0:
            slices_links[slice_id] = nx.MultiGraph()
            slices_links[slice_id].add_nodes_from(slices_links[slice_id-1].nodes(data=True))
            assert (len(slices_links[slice_id].edges()) ==0)
            #assert len(slices_links[slice_id].nodes()) >0

        if slice_id == 1+prev_slice_id and slice_id ==0:
            slices_links[slice_id] = nx.MultiGraph()

        if a not in slices_links[slice_id]:
            slices_links[slice_id].add_node(a)
        if b not in slices_links[slice_id]:
            slices_links[slice_id].add_node(b)
        slices_links[slice_id].add_edge(a,b, date=datetime_object)

    for slice_id in slices_links:
        print ("# nodes in slice", slice_id, len(slices_links[slice_id].nodes()))
        print ("# edges in slice", slice_id, len(slices_links[slice_id].edges()))
        temp = np.identity(len(slices_links[max(slices_links.keys())].nodes()))

        slices_features[slice_id] = {}
        # print(slices_links)
        for idx, node in enumerate(slices_links[slice_id].nodes()):
            # print(idx,node)
            slices_features[slice_id][node] = temp[idx]
            
    return slices_features,slices_links

In [26]:
kg= kg.sort_values(['time'])
kg = kg.reset_index()

In [9]:

links,ts = build_links(kg)


In [49]:
links

[('002837.SZ',
  'same_industry',
  '601995',
  datetime.datetime(2021, 8, 9, 4, 1, 5)),
 ('002837.SZ',
  'same_industry',
  '601995',
  datetime.datetime(2021, 8, 9, 4, 1, 5)),
 ('601728',
  'same_industry',
  '600050.SH',
  datetime.datetime(2021, 8, 9, 5, 12, 25)),
 ('601728',
  'same_industry',
  '600050.SH',
  datetime.datetime(2021, 8, 9, 5, 12, 25)),
 ('300323.SZ',
  'dispute',
  '600703.SH',
  datetime.datetime(2021, 8, 9, 6, 6, 15)),
 ('300323.SZ',
  'dispute',
  '600703.SH',
  datetime.datetime(2021, 8, 9, 6, 6, 15)),
 ('300323.SZ',
  'dispute',
  '600703.SH',
  datetime.datetime(2021, 8, 9, 6, 6, 15)),
 ('300323.SZ',
  'dispute',
  '600703.SH',
  datetime.datetime(2021, 8, 9, 6, 6, 15)),
 ('601728', 'unknown', '600905', datetime.datetime(2021, 8, 9, 8, 21, 51)),
 ('601728', 'unknown', '600905', datetime.datetime(2021, 8, 9, 8, 21, 51)),
 ('300750.SZ', 'fall', '002594.SZ', datetime.datetime(2021, 8, 9, 9, 41, 41)),
 ('300750.SZ', 'fall', '002594.SZ', datetime.datetime(2021, 8

In [54]:
relation_ls = kg['relation'].unique()

relation_d = {}
for i in range(len(relation_ls)):
    relation_d[relation_ls[i]] = i


4

In [62]:
SLICE_WEEKS = 1
START_DATE = min(ts)
MAX_DATE = max(ts)
print ("Start date", START_DATE)
slice_id = 0
trip = dict()
head = []
tail = []
relation = []
company = {}
count = 0
graph = []
# Split the set of links in order by slices to create the graphs.
for (a, b, c,time) in links:
    
    prev_slice_id = slice_id

    datetime_object = time
    if datetime_object > MAX_DATE:
        months_diff = (MAX_DATE - START_DATE).days//7  # 
    else:
        months_diff = (datetime_object - START_DATE).days//7
    slice_id = months_diff // SLICE_WEEKS
    slice_id = max(slice_id, 0) +1
    # print(slice_id,a,time)
    if a not in company:
        company[a] = count
        count+=1
    if c not in company:
        company[c] = count 
        count+=1
 
    if slice_id ==1:
        trip['Node_one'] = company[a]
        trip['Node_two'] = company[c]
        trip['Edge_type'] = relation_d[b]
        trip['Timestamp'] = slice_id
        if trip not in graph:
            graph.append(trip)

    # if slice_id == prev_slice_id +1 and slice_id >1:


print(graph)
    # if slice_id == 1+prev_slice_id and slice_id > 0:
        

Start date 2021-08-09 04:01:05
[{'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166, 'Node_two': 167, 'Edge_type': 9, 'Timestamp': 1}, {'Node_one': 166

In [65]:
graph_d = pd.DataFrame(graph)


,Node_one,Node_two,Edge_type,Timestamp
0,166,167,9,1
1,166,167,9,1
2,166,167,9,1
3,166,167,9,1
4,166,167,9,1
...,...,...,...,...
393,166,167,9,1
394,166,167,9,1
395,166,167,9,1
396,166,167,9,1


In [66]:
if graph_d['Node_one'] == 166 and graph_d['Node_two'] == 167:
    print(graph)

0      166
1      166
2      166
3      166
4      166
      ... 
393    166
394    166
395    166
396    166
397    166
Name: Node_one, Length: 398, dtype: int64

In [7]:
def build_dataset(links,ts,SLICE_MONTHS,START_DATE,MAX_DATE):
    slices_links = defaultdict(lambda : nx.MultiGraph())
    slices_features = defaultdict(lambda : {})

    print ("Start date", START_DATE)
    slice_id = 0

    # Split the set of links in order by slices to create the graphs.
    for (a, c,b, time) in links:
        prev_slice_id = slice_id

        datetime_object = time
        if datetime_object > MAX_DATE:
            months_diff = (MAX_DATE - START_DATE).days//7  # 
        else:
            months_diff = (datetime_object - START_DATE).days//7
        slice_id = months_diff // SLICE_MONTHS
        slice_id = max(slice_id, 0)
        
        if slice_id == 1+prev_slice_id and slice_id > 0:

            slices_links[slice_id] = nx.MultiGraph()
            slices_links[slice_id].add_nodes_from(slices_links[slice_id-1].nodes(data=True))
            assert (len(slices_links[slice_id].edges()) ==0)
            #assert len(slices_links[slice_id].nodes()) >0

        if slice_id == 1+prev_slice_id and slice_id ==0:
            slices_links[slice_id] = nx.MultiGraph()

        if a not in slices_links[slice_id]:
            slices_links[slice_id].add_node(a)
        if b not in slices_links[slice_id]:
            slices_links[slice_id].add_node(b)
        slices_links[slice_id].add_edge(a,b, date=datetime_object)
        
        
       

    for slice_id in slices_links:
        print ("# nodes in slice", slice_id, len(slices_links[slice_id].nodes()))
        print ("# edges in slice", slice_id, len(slices_links[slice_id].edges()))
        temp = np.identity(len(slices_links[max(slices_links.keys())].nodes()))

        slices_features[slice_id] = {}
        # print(slices_links)
        for idx, node in enumerate(slices_links[slice_id].nodes()):
            print(idx,node)
            slices_features[slice_id][node] = temp[idx]
            
    return slices_features,slices_links

In [10]:
SLICE_WEEKS = 1
START_DATE = min(ts)
MAX_DATE = max(ts)

In [8]:
slices_features,slices_links = build_dataset(links,ts,SLICE_WEEKS,START_DATE,MAX_DATE)
# slices_links_remap, slices_features_remap = remap(slices_links, slices_features)

NameError: name 'links' is not defined

In [ ]:
 print(slices_links[slice_id].edges())


# remap

In [52]:
from scipy.sparse import csr_matrix

def remap(slices_graph, slices_features):
    all_nodes = []
    for slice_id in slices_graph:
        assert len(slices_graph[slice_id].nodes()) == len(slices_features[slice_id])
        all_nodes.extend(slices_graph[slice_id].nodes())
    all_nodes = list(set(all_nodes))
    # print ("Total # nodes", len(all_nodes), "max idx", max(all_nodes))
    print ("Total # nodes", len(all_nodes))
    ctr = 0
    node_idx = {}
    idx_node = []
    for slice_id in slices_graph:
        for node in slices_graph[slice_id].nodes():
            if node not in node_idx:
                node_idx[node] = ctr
                idx_node.append(node)
                ctr += 1
    slices_graph_remap = []
    slices_features_remap = []
    for slice_id in slices_graph:
        G = nx.MultiGraph()
        for x in slices_graph[slice_id].nodes():
            G.add_node(node_idx[x])
        for x in slices_graph[slice_id].edges(data=True):
            G.add_edge(node_idx[x[0]], node_idx[x[1]], date=x[2]['date'])
        assert (len(G.nodes()) == len(slices_graph[slice_id].nodes()))
        assert (len(G.edges()) == len(slices_graph[slice_id].edges()))
        slices_graph_remap.append(G)
    
    for slice_id in slices_features:
        features_remap = []
        for x in slices_graph_remap[slice_id].nodes():
            features_remap.append(slices_features[slice_id][idx_node[x]])
            #features_remap.append(np.array(slices_features[slice_id][idx_node[x]]).flatten())
        features_remap = csr_matrix(np.squeeze(np.array(features_remap)))
        slices_features_remap.append(features_remap)
    return (slices_graph_remap, slices_features_remap)

# overall


In [22]:
subgraph_set = [subgraph_compete,subgraph_increase_holding,subgraph_be_invested,subgraph_reduce_holding,subgraph_subordinate,
subgraph_be_increased_holding,subgraph_dispute,
subgraph_be_supplied,subgraph_same_industry, subgraph_rise, subgraph_cooperate,
subgraph_fall, subgraph_supply, subgraph_be_reduced_holding,subgraph_superior, subgraph_invest, 
]

In [47]:
subgraph_be_supplied

,index,time,head_code,tail_code
0,748,2021-11-18 15:41:37,300093.SZ,002129.SZ
1,1740,2021-12-02 17:24:25,688567,603799.SH
2,1353,2021-12-21 18:14:00,600536.SH,603897.SH
3,2726,2021-12-21 18:14:01,600536.SH,603897.SH
4,2586,2022-01-24 21:49:10,002272.SZ,601615.SH
5,283,2022-01-30 12:27:25,300750.SZ,300014.SZ
6,1529,2022-02-20 11:06:05,300750.SZ,600848.SH
7,3134,2022-02-20 13:06:04,300750.SZ,600848.SH


same_industry 197
rise 430
compete 19
unknown 1464
cooperate 481
increase_holding 21
fall 253
supply 100
be_reduced_holding 45
be_invested 20
reduce_holding 1
superior 49
be_increased_holding 23
subordinate 3
invest 18
dispute 17
be_supplied 8

In [20]:
subgraph_set_str = ['compete','increase_holding','be_invested','reduce_holding','subordinate','be_increased_holding','dispute','be_supplied',
'same_industry', 'rise',  'cooperate',
'fall', 'supply', 'be_reduced_holding' , 'superior', 'invest', 
]

In [20]:
# 'same_industry', 'rise', 'cooperate','fall','supply',


In [26]:
start_date = []
end_date = []
for i in range (0,len(subgraph_set)):
    if len(subgraph_set) != len(subgraph_set_str):
        print('wrong!!!')
    # print('start generate the graph:',subgraph_set_str[i])
    links,ts = build_links(subgraph_set[i])
    start_date.append(min(ts))
    end_date.append(max(ts))


start generate the graph: compete
start generate the graph: increase_holding
start generate the graph: be_invested
start generate the graph: reduce_holding
start generate the graph: subordinate
start generate the graph: be_increased_holding
start generate the graph: dispute
start generate the graph: be_supplied
start generate the graph: same_industry
start generate the graph: rise
start generate the graph: cooperate
start generate the graph: fall
start generate the graph: supply
start generate the graph: be_reduced_holding
start generate the graph: superior
start generate the graph: invest


In [50]:
START_DATE = max(start_date)
MAX_DATE = min(end_date) 
print('START_DATE ',START_DATE)
print('END_DATE',MAX_DATE)
SLICE_WEEKS = 5

START_DATE  2021-12-08 19:56:00
END_DATE 2022-02-11 19:36:42


In [51]:
for i in range (0,len(subgraph_set)):
    if len(subgraph_set) != len(subgraph_set_str):
        print('wrong!!!')
    print('start generate the graph:',subgraph_set_str[i])
    links,ts = build_links(subgraph_set[i])
    slices_features,slices_links = subgraph(links,ts,SLICE_WEEKS,START_DATE,MAX_DATE)
    slices_links_remap, slices_features_remap = remap(slices_links, slices_features)
    dir = 'FinDyKG/'+ subgraph_set_str[i]+'_sencond_round'
    if not os.path.isdir(dir):
        os.makedirs(dir)
    np.savez(dir+'/graphs.npz', graph=slices_links_remap,protocol=2)
    np.savez(dir+'/features.npz', feats=slices_features_remap,protocol=2)
    

start generate the graph: compete
Start date 2021-12-08 19:56:00
# nodes in slice 0 14
# edges in slice 0 19
# nodes in slice 1 16
# edges in slice 1 11
Total # nodes 16 max idx 601988
start generate the graph: increase_holding
Start date 2021-12-08 19:56:00
# nodes in slice 0 102
# edges in slice 0 73
# nodes in slice 1 110
# edges in slice 1 4
Total # nodes 110 max idx 688023.SH
start generate the graph: be_invested
Start date 2021-12-08 19:56:00
# nodes in slice 0 24
# edges in slice 0 12
# nodes in slice 1 34
# edges in slice 1 8
Total # nodes 34 max idx 688779
start generate the graph: reduce_holding
Start date 2021-12-08 19:56:00
# nodes in slice 0 73
# edges in slice 0 54
# nodes in slice 1 75
# edges in slice 1 1
Total # nodes 75 max idx 688776
start generate the graph: subordinate
Start date 2021-12-08 19:56:00
# nodes in slice 0 2
# edges in slice 0 1
# nodes in slice 1 6
# edges in slice 1 2
Total # nodes 6 max idx 600482.SH
start generate the graph: be_increased_holding
Sta

In [ ]:
# slices_links_remap, slices_features_remap = remap(slices_links, slices_features)

Total # nodes 299 max idx 876


In [ ]:
# np.savez('FinDyKG/cooperate_graphs_new.npz', graph=slices_links_remap,protocol=2)
# np.savez('FinDyKG/cooperate_features_new.npz', feats=slices_features_remap,protocol=2)

# 备用

In [ ]:
with open('supply_graphs.npz', 'wb') as f:
    np.savez(f, graph=slices_links_remap)

In [ ]:
# to id 
company_ls_1 = list(kg_df['head'])
company_ls_2 = list(kg_df['tail'])
company_ls = company_ls_1 + company_ls_2

In [ ]:
company_ls = list(set(company_ls))

In [ ]:
company_dict = dict(company_ls)

ValueError: dictionary update sequence element #0 has length 9; 2 is required

In [ ]:
company_ls


NameError: name 'company_ls' is not defined